<img src="https://heig-vd.ch/docs/default-source/doc-global-newsletter/2020-slim.svg" alt="HEIG-VD Logo" width="100" align="right" /> 

# Cours TAL - Laboratoire 2<br/>*POS taggers* pour le français dans spaCy et NLTK

**Objectif**

Comparer l'étiqueteur morphosyntaxique français prêt-à-l'emploi de spaCy avec deux étiqueteurs entraînés, l'un dans spaCy et l'autre dans NLTK.

## 1. Installation et test de spaCy

La boîte à outils spaCy est une librairie Python *open source* pour le TAL, dédiée à un usage en production. Les documents suivants vous seront utiles :
* comment [installer](https://spacy.io/usage) spaCy
* comment [télécharger un modèle](https://spacy.io/usage/models) pour une langue donnée (on appelle ces modèles des *trained pipelines* car ils enchaînent plusieurs traitements)
* comment faire les [premiers pas](https://spacy.io/usage/spacy-101) dans l'utilisation de spaCy

Veuillez installer spaCy, puis la *pipeline* pour le français appelée `fr_core_news_sm`.  Si vous utilisez *conda*, installez spaCy dans l'environnement du cours TAL.

In [1]:
import spacy
from markdown_it.rules_block import reference

nlp = spacy.load("fr_core_news_sm")  # charge la pipeline
import tqdm  # permet l'affichage d'une barre de progression

**1a.** Une pipeline effectue un ensemble de traitements d'un texte en lui ajoutant des annotations.  Les traitements effectués par la pipeline `fr_core_news_sm` sont [documentés ici](https://spacy.io/models/fr#fr_core_news_sm).  La liste des traitements d'une pipeline figure dans son attribut `.pipe_names`.  On peut activer ou désactiver un traitement T avec, respectivement, les méthodes `.disable_pipe(T)` et `.enable_pipe(T)` appliquées à la pipeline.

* Veuillez afficher les traitements disponibles dans la pipeline `fr_core_news_sm` chargée ci-dessus sous le nom de `nlp` .
* Veuillez désactiver tous les traitements sauf `tok2vec` et `morphologizer` (on fait cela pour accélerer le traitement).
* Vérifiez que la désactivation a bien fonctionné en affichant les traitements activés.

In [2]:
# Veuillez écrire votre code ici.

print("=== Liste des traitements disponibles")
for pipe in nlp.pipe_names:
    print(pipe)

for pipe in nlp.pipe_names:
    if pipe != "tok2vec" and pipe != "morphologizer":
        nlp.disable_pipe(pipe)

print("=== Liste des traitements activés")
for pipe in nlp.pipe_names:
    print(pipe)

=== Liste des traitements disponibles
tok2vec
morphologizer
parser
attribute_ruler
lemmatizer
ner
=== Liste des traitements activés
tok2vec
morphologizer


In [3]:
from spacy.lang.fr.examples import sentences

**1b.** L'objet `sentences` chargé ci-dessus contient une liste de phrases en français. 

* Veuillez afficher les deux premières phrases de `sentences`.
* Veuillez analyser chacune de ces deux phrases avec la pipeline `nlp` puis afficher chaque token et son POS tag.
    * indication : aidez-vous de la [documentation](https://spacy.io/models/fr#fr_core_news_sm) de `fr_core_news_sm`
    * consigne d'affichage : indiquer le tag entre crochets après chaque token, comme ceci : Les \[DET\] robots \[NOUN\] ...
    * note : la documentation détaillée du POS tagging dans spaCy est [disponible ici](https://spacy.io/usage/linguistic-features)
* Veuillez commenter la tokenisation et les POS tags observés : vous semblent-ils corrects pour les deux phrases ?

In [4]:
# Veuillez écrire votre code et votre commentaire ici.
print(f"Premières phrases: {sentences[:2]}")

result = []
for sentence in sentences[:2]:
    doc = nlp(sentence)
    res = []
    for token in doc:
        res.append(f'{token.text} [{token.pos_}]')

    result.append(res)

print("=== Résultat")
for res in result:
    print(' '.join(res))

Premières phrases: ['Apple cherche à acheter une start-up anglaise pour 1 milliard de dollars', "Les voitures autonomes déplacent la responsabilité de l'assurance vers les constructeurs"]
=== Résultat
Apple [NOUN] cherche [NOUN] à [ADP] acheter [VERB] une [DET] start [NOUN] - [NOUN] up [ADJ] anglaise [NOUN] pour [ADP] 1 [NUM] milliard [NOUN] de [ADP] dollars [NOUN]
Les [DET] voitures [NOUN] autonomes [ADJ] déplacent [ADV] la [DET] responsabilité [NOUN] de [ADP] l' [DET] assurance [NOUN] vers [ADP] les [DET] constructeurs [NOUN]


> On constate que le mot start-up a été taggé en trois parties, comportant le trait d'union comme nom rejoignant les deux parties du mot. Le up de start-up est du coup considéré comme un adjectif.
>
> On peut supposer que le tagger a une valeur par défaut pour les mots inconnus, ou que l'entrainement ayant été fait avec les mots français le modèle soit moins performant avec les anglicismes.
>
> Le tagging de la deuxième phrase semble correct.

## 2. Prise en main des données

Les données sont fournies dans un format tabulaire dans l'archive `UD_French-GSD.zip` sur Cyberlearn.  Elles sont basées sur les données fournies par le projet [Universal Dependencies](https://github.com/UniversalDependencies/UD_French-GSD).  Leur format, appelé CoNLL-U, est [documenté ici](https://universaldependencies.org/format.html).  Veuillez placer les trois fichiers contenus dans l'archive dans un sous-dossier de ce notebook nommé `spacy_data`.

Les trois fichiers contiennent des phrases en français annotées avec les POS tags :
* le fichier `fr-ud-train.conllu` est destiné à l'entraînement
* le fichier `fr-ud-dev.conllu` est destiné aux tests préliminaires et aux réglages des paramètres
* le fichier `fr-ud-test.conllu` est destiné à l'évaluation finale.

**2a.** En inspectant les fichiers avec un éditeur texte, veuillez déterminer dans quelle colonne se trouvent les *tokens* des textes originaux, et dans quelle colonne se trouvent leurs étiquettes morpho-syntaxiques correctes (*POS tags*).  Que contient la troisième colonne ?

- On compte la première colonne comme étant celle des identifiants.
- La deuxième colonne comprend les tokens des textes originaux.
- La quatrième colonne comprend les étiquettes morpho-syntaxiques correctes.
- La troisième colonne contient le lemme des tokens.

**2b.** Veuillez convertir les trois fichiers de données en des fichiers binaires utilisables par spaCy, en utilisant la [commande 'convert' fournie par spaCy](https://spacy.io/api/cli#convert).  La commande est donnée ci-dessous, le premier dossier `./input_data` contient les 3 fichiers `.conllu` et le dossier `./spacy-data` contiendra les 3 résultats.

* Veuillez exécuter la commande de conversion.
* Combien de phrases environ (à 10 phrases près) contient chaque fichier (*train*, *dev*, *test*) ?  Observez la commande et son résultat pour répondre.

In [5]:
!python -m spacy convert ./input_data ./spacy_data --converter conllu  --n-sents 10 --lang fr

ℹ Grouping every 10 sentences into a document.
✔ Generated output file (148 documents): spacy_data/fr-ud-dev.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (42 documents): spacy_data/fr-ud-test.spacy
ℹ Grouping every 10 sentences into a document.
✔ Generated output file (1456 documents):
spacy_data/fr-ud-train.spacy


In [29]:
# Veuillez indiquer les nombres de phrases ici.
INPUT_FILE_DEV = "./input_data/fr-ud-dev.conllu"
INPUT_FILE_TEST = "./input_data/fr-ud-test.conllu"
INPUT_FILE_TRAIN = "./input_data/fr-ud-train.conllu"

def count_sentences(file):
    with open(file, 'r') as f:
        return len([line for line in f if line.strip() != ""])

print(f"Nombre de phrases dans le fichier de développement: {count_sentences(INPUT_FILE_DEV)}")
print(f"Nombre de phrases dans le fichier de test: {count_sentences(INPUT_FILE_TEST)}")
print(f"Nombre de phrases dans le fichier d'entraînement: {count_sentences(INPUT_FILE_TRAIN)}")

Nombre de phrases dans le fichier de développement: 36830
Nombre de phrases dans le fichier de test: 10298
Nombre de phrases dans le fichier d'entraînement: 366371


**2c**. Les données des fichiers convertis peuvent être chargées dans un objet de type `DocBin`.  Dans notre cas, un tel objet contient un ensemble de documents, chacun contenant 10 phrases.  Chaque document est un objet de type `Doc`.  Le code donné ci-dessous vous permet de charger les données de test et vous montre comment les afficher.

* Veuillez stocker la première phrase des données de test dans une variable nommée `premiere_phrase_test`.
* Veuillez afficher cette phrase, ainsi que son type dans spaCy.

In [7]:
from spacy.tokens import DocBin
from spacy.tokens import Doc

FILE_DEV = "./input_data/fr-ud-dev.conllu"
FILE_TEST = "./input_data/fr-ud-test.conllu"
FILE_TRAIN = "./input_data/fr-ud-train.conllu"

test_data = DocBin().from_disk("./spacy_data/fr-ud-test.spacy")
# Exemple d'utilisation (afficher toutes les phrases)
# for doc in test_data.get_docs(nlp.vocab):
#     for sent in doc.sents:
#         print(sent)

In [8]:
# Veuillez écrire votre code ici.

first_doc = next(test_data.get_docs(nlp.vocab))
premiere_phrase_test = next(first_doc.sents)
print(f"Première phrase des données de test: {premiere_phrase_test.text}")
print(f"Type de la première phrase: {type(premiere_phrase_test)}")

Première phrase des données de test: Je sens qu'entre ça et les films de médecins et scientifiques fous que nous avons déjà vus, nous pourrions emprunter un autre chemin pour l'origine.
Type de la première phrase: <class 'spacy.tokens.span.Span'>


## 3. Évaluation du POS tagger français de la pipeline `fr_core_news_sm`

**3a.** Veuillez effectuer le *POS tagging* avec spaCy de la `premiere_phrase_test` et afficher les résultats dans le format demandé au (1b).  Indication : convertissez la `premiere_phrase_test` dans un objet de type `Doc` en lui appliquant la méthode `.as_doc()`.  Cet objet peut être ensuite traité par la pipeline `nlp`.

In [9]:
# Veuillez écrire votre code ici.

sentence_doc = nlp(premiere_phrase_test.as_doc().text)
res_pipeline = []
for token in sentence_doc:
    res_pipeline.append(f'{token.text} [{token.pos_}]')

print(f"Résultat de la pipeline sur la première phrase: {' '.join(res_pipeline)}")

Résultat de la pipeline sur la première phrase: Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [PRON] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT]


**3b.** Veuillez afficher les tags corrects de `premiere_phrase_test`, puis comparez-les visuellement les tags trouvés automatiquement au (3a).  Quelles différences trouvez-vous ?

In [10]:
# Veuillez écrire votre réponse ici.

res_model = []
for token in premiere_phrase_test:
    res_model.append(f'{token.text} [{token.pos_}]')

print(f"Tags corrects de la première phrase: {' '.join(res_model)}")


Tags corrects de la première phrase: Je [PRON] sens [VERB] qu' [SCONJ] entre [ADP] ça [PRON] et [CCONJ] les [DET] films [NOUN] de [ADP] médecins [NOUN] et [CCONJ] scientifiques [NOUN] fous [ADJ] que [PRON] nous [PRON] avons [AUX] déjà [ADV] vus [VERB] , [PUNCT] nous [PRON] pourrions [VERB] emprunter [VERB] un [DET] autre [ADJ] chemin [NOUN] pour [ADP] l' [DET] origine [NOUN] . [PUNCT]


In [11]:
print("Différences entre la pipeline et le modèle:")
bad_count = 0
for i, (token, token2) in enumerate(zip(res_pipeline, res_model)):
    if token != token2:
        bad_count += 1
        print(f"Token {i + 1}: {token} != {token2}")

print(f"Précision: {1 - bad_count / len(res_model)}")

Différences entre la pipeline et le modèle:
Token 13: fous [PRON] != fous [ADJ]
Précision: 0.9655172413793104


> On constate une différence sur le token numéro 13 ("fous"), qui est taggé comme un pronom par la pipeline, alors que les données de tests en font mention en tant qu'adjectif ce qui bien entendu juste. On parle bien des scientifiques qui sont fous.

In [12]:
from spacy.scorer import Scorer
from spacy.training import Example

In [13]:
scorer = Scorer()

**3c.** Au lieu de compter manuellement combien de tags sont différents entre la référence et le résultat de la pipeline `nlp`, vous allez utiliser la classe `Scorer` de spaCy.  Une instance de cette classe permet de calculer les scores d'une liste d'objets de type `Exemple`, en fonction des annotations disponibles dans les objets.  Un objet de type `Exemple` contient deux objets de type `Doc`, l'un avec les annotations correctes et l'autre avec les annotations produites par une pipeline.  La [documentation de la méthode](https://spacy.io/api/scorer#score) `Scorer.score(..)` vous sera utile. 

* Veuillez calculer la justesse (*accuracy*) du *POS tagging* de `premiere_phrase_test`. 
* Veuillez justifier la valeur du score obtenu en utilisant votre réponse du (3b).

In [25]:
# Veuillez écrire votre code ici.

score = scorer.score([Example(sentence_doc, premiere_phrase_test.as_doc())])
print(f"POS Accuracy: {score.get('pos_acc')}")

POS Accuracy: 0.9655172413793104


> Nous obtenons donc une précision du tagging à 96.5%, ce qui correspond à ce qui a été observé précédemment. Un seul token n'était pas correctement taggé, et donc sur l'ensemble de tokens de la phrase la valeur correspond.

**3d.** Veuillez calculer la précision du *POS tagging* de la pipeline `nlp` sur toutes les données de test présentes dans `test_data`.  Comment se compare le score obtenu avec celui mentionné [dans la documentation](https://spacy.io/models/fr#fr_core_news_sm) du modèle `fr_core_news_sm` ?

In [15]:
# Veuillez écrire votre code ici, suivi de votre réponse à la question.

score_docs = scorer.score([Example(nlp(doc.text), doc) for doc in test_data.get_docs(nlp.vocab)])
print(f"POS Accuracy over whole documents: {score_docs.get('pos_acc')}")

sentences = [sentence for doc in test_data.get_docs(nlp.vocab) for sentence in doc.sents]
score_sentences = scorer.score([Example(nlp(sentence.as_doc().text), sentence.as_doc()) for sentence in sentences])
print(f"POS Accuracy over sentences: {score_sentences.get('pos_acc')}")

POS Accuracy over whole documents: 0.8809902922594894
POS Accuracy over sentences: 0.8806246147524142


> Nous avons en premier lieu comparé la précision sur l'ensemble des documents, en demandant à la pipeline de traiter en une fois l'ensemble des phrases de chaque document. Nous constatons une précision à 0.88, nous avons ensuite chercher à comparer la précision si le tagging était fait phrase par phrase sans avoir le contexte (comme effectué dans les exercices précédents). La précision dans ce cas est pratiquement égale, ce qui nous semble toujours très correct étant donné que dans certains cas le contexte peut aider à déterminer le bon tag.
>
> La documentation du modèle préconise une précision sur le tagging de 0.96, ce qui est légèrement supérieur à ce que nous observons ici. Probablement que le modèle fourni a été entrainé sur un ensemble de données qui n'est pas contextuellement similaire aux données de test que nous utilisons ici.

## 4. Entraîner puis évaluer un nouveau POS tagger français dans spaCy

Le but de cette partie est d'entraîner une pipeline spaCy pour le français sur les données de `fr-ud-train.conllu`, puis de comparer le modèle obtenu avec le modèle prêt-à-l'emploi testé au point précédent.  Les [instructions d'entraînement](https://spacy.io/usage/training#quickstart) de spaCy vous montrent comment entraîner une pipeline avec un POS tagger.

**4a.** Paramétrage de l'entraînement :
* générez un fichier de départ grâce à [l'interface web](https://spacy.io/usage/training#quickstart), en indiquant que vous voulez seulement un POS tagger dans la pipeline ;
* sauvegardez le code généré par spaCy dans un fichier local `base_config.cfg` ;
* générez un fichier `config.cfg` sur votre ordinateur en exécutant la ligne de commande suivante. 

In [16]:
!python -m spacy init fill-config base_config.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


Enfin, veuillez effectuer l'entraînement avec la ligne de commande suivante.  Faites plusieurs essais, d'abord avec un petit nombre d'époques, pour estimer le temps nécessaire et observer les messages affichés.  Puis augmentez progressivement le nombre d'époques.  Quel est le critère qui vous permet de décider que vous avez un nombre suffisant d'époques ?  Dans quel dossier se trouve le meilleur modèle ?

In [17]:
# Dernier résultat dans myPOStagger-5epoch
# !python -m spacy train config.cfg --output './myPOStagger-5epoch' --paths.train ./spacy_data/fr-ud-train.spacy --paths.dev ./spacy_data/fr-ud-dev.spacy --verbose

Avec `max_epoch=1`

```
E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE
---  ------  ------------  -----------  -------  ------
  0       0          0.00       211.77    36.34    0.36
  0     200        315.78     10403.58    90.38    0.90
  0     400        287.59      4500.03    91.63    0.92
  0     600        222.64      3447.72    92.14    0.92
  0     800        217.77      3472.31    92.58    0.93
  0    1000        190.27      3012.89    92.61    0.93
  0    1200        184.36      2921.70    92.90    0.93
  0    1400        171.64      2746.66    93.02    0.93
```

Avec `max_epoch=5`

```
E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE
---  ------  ------------  -----------  -------  ------
  0       0          0.00       211.77    36.34    0.36
  0     200        315.78     10403.58    90.38    0.90
  0     400        287.59      4500.03    91.63    0.92
  0     600        222.64      3447.72    92.14    0.92
  0     800        217.77      3472.31    92.58    0.93
  0    1000        190.27      3012.89    92.61    0.93
  0    1200        184.36      2921.70    92.90    0.93
  0    1400        171.64      2746.66    93.02    0.93
  1    1600        147.59      2232.49    93.10    0.93
  1    1800        140.07      2025.26    93.13    0.93
  1    2000        155.89      2274.63    93.22    0.93
  1    2200        147.96      2143.40    93.26    0.93
  1    2400        148.56      2133.26    93.39    0.93
  1    2600        150.10      2141.56    93.20    0.93
  1    2800        152.66      2182.57    93.46    0.93
  2    3000        131.79      1809.59    93.36    0.93
  2    3200        122.10      1576.16    93.27    0.93
  2    3400        134.89      1725.80    93.49    0.93
  2    3600        131.55      1643.79    93.47    0.93
  2    3800        126.99      1564.77    93.42    0.93
  2    4000        133.31      1665.34    93.42    0.93
  2    4200        141.77      1755.45    93.55    0.94
  3    4400        139.83      1696.87    93.53    0.94
  3    4600        101.02      1149.63    93.57    0.94
  3    4800        116.95      1310.59    93.61    0.94
  3    5000        127.54      1422.67    93.49    0.93
  3    5200        126.94      1419.86    93.60    0.94
  3    5400        125.46      1401.61    93.52    0.94
  3    5600        123.99      1371.43    93.57    0.94
  3    5800        133.30      1439.02    93.69    0.94
  4    6000        111.11      1149.63    93.63    0.94
  4    6200        105.39      1048.16    93.56    0.94
  4    6400        112.71      1114.82    93.63    0.94
  4    6600        121.72      1216.46    93.67    0.94
  4    6800        121.94      1222.12    93.68    0.94
  4    7000        121.71      1226.58    93.60    0.94
  4    7200        114.93      1125.15    93.61    0.94
```

Avec `max_epoch=0` (arrêt automatique après 1600 étapes sans amélioration, configuration par défaut)

```
E    #       LOSS TOK2VEC  LOSS TAGGER  TAG_ACC  SCORE
---  ------  ------------  -----------  -------  ------
  0       0          0.00       211.77    36.34    0.36
  0     200        315.78     10403.58    90.38    0.90
  0     400        287.59      4500.03    91.63    0.92
  0     600        222.64      3447.72    92.14    0.92
  0     800        217.77      3472.31    92.58    0.93
  0    1000        190.27      3012.89    92.61    0.93
  0    1200        184.36      2921.70    92.90    0.93
  0    1400        171.64      2746.66    93.02    0.93
  1    1600        147.59      2232.49    93.10    0.93
  1    1800        140.07      2025.26    93.13    0.93
  1    2000        155.89      2274.63    93.22    0.93
  1    2200        147.96      2143.40    93.26    0.93
  1    2400        148.56      2133.26    93.39    0.93
  1    2600        150.10      2141.56    93.20    0.93
  1    2800        152.66      2182.57    93.46    0.93
  2    3000        131.79      1809.59    93.36    0.93
  2    3200        122.10      1576.16    93.27    0.93
  2    3400        134.89      1725.80    93.49    0.93
  2    3600        131.55      1643.79    93.47    0.93
  2    3800        126.99      1564.77    93.42    0.93
  2    4000        133.31      1665.34    93.42    0.93
  2    4200        141.77      1755.45    93.55    0.94
  3    4400        139.83      1696.87    93.53    0.94
  3    4600        101.02      1149.63    93.57    0.94
  3    4800        116.95      1310.59    93.61    0.94
  3    5000        127.54      1422.67    93.49    0.93
  3    5200        126.94      1419.86    93.60    0.94
  3    5400        125.46      1401.61    93.52    0.94
  3    5600        123.99      1371.43    93.57    0.94
  3    5800        133.30      1439.02    93.69    0.94
  4    6000        111.11      1149.63    93.63    0.94
  4    6200        105.39      1048.16    93.56    0.94
  4    6400        112.71      1114.82    93.63    0.94
  4    6600        121.72      1216.46    93.67    0.94
  4    6800        121.94      1222.12    93.68    0.94
  4    7000        121.71      1226.58    93.60    0.94
  4    7200        114.93      1125.15    93.61    0.94
  5    7400        106.20      1017.87    93.62    0.94
```


> Le critère principal pour déterminer que le nombre d'epochs est suffisant est si l'amélioration stagne ou si la précision n'augmente plus.
> La configuration par défaut utilise un nombre illimité d'epoch mais arrete si le modèle stagne depuis 1600 étapes. Cela permet d'éviter de devoir déterminer le nombre d'épochs de façon empirique.
>
> Le meilleur modèle d'une série d'entrainement est dans le sous-dossier `model-best` du dossier cible.
>
> Dans notre cas, le modèle s'arrête après 5 épochs, avec une précision de 0.94.


**4b.**  Veuillez charger le meilleur modèle (pipeline) dans la variable `nlp2` et afficher la *POS tagging accuracy* sur le corpus de test.  Le composant de la pipeline étant un *POS tagger*, vous devrez évaluer la propriété *tag_acc*. 

In [18]:
# Veuillez écrire votre code ici.

nlp2 = spacy.load("./myPOStagger-5epoch/model-best")
score = scorer.score([Example(nlp2(doc.text), doc) for doc in list(test_data.get_docs(nlp.vocab))])

print(f"Précision du POS tagging: {score.get('tag_acc')}")

Précision du POS tagging: 0.9341969977380219


## 5. Entraîner puis évaluer un POS tagger pour le français dans NLTK

Le but de cette partie est d'utiliser le POS tagger appelé *Averaged Perceptron* fourni par NLTK, en l'entraînant pour le français sur les mêmes données que ci-dessus, importées cette fois-ci avec NLTK.  Pour une introduction au POS tagging avec NLTK, voir le [Chapitre 5.1 du livre NLTK](http://www.nltk.org/book/ch05.html).

Remarques :
* pour l'anglais, des taggers pré-entraînés sont disponibles dans NLTK ;
* pour appliquer un tagger existant, on écrit `nltk.pos_tag(sentence)` où `sentence` est une liste de tokens et on obtient des paires (token, TAG) ;
* l'implémentation de *Averaged Perceptron* a été faite par [Mathew Honnibal de Explosion.AI](https://explosion.ai/blog/part-of-speech-pos-tagger-in-python), la société qui a créé spaCy.

**5a.** Veuillez charger les données d'entraînement et celles de test grâce à la classe `ConllCorpusReader` de NLTK.  [La documentation de cette classe](https://www.nltk.org/api/nltk.corpus.reader.conll.html#nltk.corpus.reader.conll.ConllCorpusReader) vous montrera comment indiquer les colonnes qui contiennent les tokens ('words') et les tags corrects ('pos').  Une fois les données chargées dans une variable, vous pouvez accéder aux phrases et aux tags avec la méthode `.tagged_sents()`.

In [19]:
from nltk.corpus.reader.conll import ConllCorpusReader

In [20]:
# Veuillez écrire votre code ici.

train = ConllCorpusReader("./input_data", "fr-ud-train.conllu", ("ignore", "words", "ignore", "pos"), separator="\t")
test = ConllCorpusReader("./input_data", "fr-ud-test.conllu", ("ignore", "words", "ignore", "pos"), separator="\t")

sentences_train = train.tagged_sents()
sentences_test = test.tagged_sents()

print(sentences_test[0])


[('Je', 'PRON'), ('sens', 'VERB'), ("qu'", 'SCONJ'), ('entre', 'ADP'), ('ça', 'PRON'), ('et', 'CCONJ'), ('les', 'DET'), ('films', 'NOUN'), ('de', 'ADP'), ('médecins', 'NOUN'), ('et', 'CCONJ'), ('scientifiques', 'NOUN'), ('fous', 'ADJ'), ('que', 'PRON'), ('nous', 'PRON'), ('avons', 'AUX'), ('déjà', 'ADV'), ('vus', 'VERB'), (',', 'PUNCT'), ('nous', 'PRON'), ('pourrions', 'VERB'), ('emprunter', 'VERB'), ('un', 'DET'), ('autre', 'ADJ'), ('chemin', 'NOUN'), ('pour', 'ADP'), ("l'", 'DET'), ('origine', 'NOUN'), ('.', 'PUNCT')]


**5b.** Pour entraîner un POS tagger du type Averaged Perceptron, vous utiliserez le sous-module `nltk.tag.perceptron` du [module NLTK contenant les taggers](http://www.nltk.org/api/nltk.tag.html).  Les fonctions d'entraînement et de test sont documentées dans ce module.  Après l'entraînement, le réseau de neurones est enregistré dans un fichier `.pickle`, qui est écrasé à chaque entraînement si vous n'en faites pas une copie.  On peut également lire un fichier `.pickle` dans un tagger.

Veuillez écrire le code pour entraîner le POS tagger sur les données d'entraînement.  Comme au (4), pensez augmenter graduellement le nombre d'époques (appelées 'itérations' dans NLTK).

Combien de temps prend l'entraînement ?  Quelle est la taille du fichier enregistré ?

In [21]:
# NOTE: NLTK a retiré la gestion des fichiers pickle pour un stockage sous format JSON il y a 8 mois,
#       il n'est plus possible de sauvegarder directement le fichier pickle du a un bug dans la librairie,
#       on doit donc passer par la méthode save_to_json pour sauvegarder le modèle en format JSON,
#       mais elle ne marche pas. On force le requirement à une version qui fonctionne...
!python -m pip install 'nltk==3.8.1'

import os
# import nltk
# nltk.download('averaged_perceptron_tagger') # à exécuter la première fois
from nltk.tag.perceptron import PerceptronTagger

In [22]:
ptagger = PerceptronTagger(load=False)

In [23]:
# Veuillez écrire votre code ici.

#ptagger.train(sentences_train, "./nltkPOStagger/model-2epoch.pickle", nr_iter=2)
#ptagger.train(sentences_train, "./nltkPOStagger/model-5epoch.pickle", nr_iter=5)
ptagger.train(sentences_train, "./nltkPOStagger/model-10epoch.pickle", nr_iter=10)

> L'entrainement pour deux épochs a pris 20 secondes, le fichier enregistré fait 5.2 Mo.
>
> L'entrainement pour cinq épochs a pris 43 secondes, le fichier enregistré fait 6.3 Mo.
>
> L'entrainement pour dix épochs a pris 1 minute 25 secondes, le fichier enregistré fait 6.9 Mo.

**5c.** Veuillez évaluer le tagger sur les données de test et afficher le taux de correction.

In [28]:
# Veuillez écrire votre code ici.

print(f"Précision du POS tagging avec NLTK: {ptagger.accuracy(sentences_test)}")

Précision du POS tagging avec NLTK: 0.9601864439697029


## 6. Conclusion

Veuillez remplir le tableau suivant avec les scores obtenus et discuter brièvement comment se comparent les trois POS taggers sur ces données de test.

| spaCy (partie 3) | spaCy (partie 4) | NLTK (partie 5) | 
|------------------|------------------|-----------------|
| tagger fourni    | tagger entraîné  | tagger entraîné |
| 0.8810           | 0.9342           | 0.9602          |

> On constate avec ces scores que le tagger entrainé avec NLTK obtient les meilleurs résultats, suivi par le tagger entrainé avec spaCy sur les données fournies similaires aux données de test. Le tagger avec le modèle fourni par spaCy obtient les moins bons résultats, ce qui est logique étant donné que le modèle a été conçu sur un ensemble de données qui ont un contexte ou un sens différent des données que nous avons utilisé pour le test.
>
> En entrainant notre propre modèle, on est donc capable de créer un tagger qui se conforme plus à la sémantique, le contexte et le sens du corpus que nous souhaitons étudier. Alors qu'utiliser un modèle fourni aura des limitations dans ce qui est possible d'atteindre en termes de précision. Il faut toutefois faire attention à ne pas entrainer un modèle sur un corpus trop spécifique, car nous pourrions tomber dans un cas classique d'overfitting où le modèle ne sera plus capable de généraliser sur des données qu'il n'a pas vu.


**Fin du Labo.** Veuillez nettoyer ce notebook en gardant seulement les résultats désirés, l'enregistrer, et le soumettre comme devoir sur Cyberlearn.